# Analisi Scientifica: Impatto del Neighborhood Size sui Modelli NCA

Questo notebook esegue un'analisi completa e scientifica per determinare se ha senso utilizzare un `neighborhood_size` maggiore di 3, e quali sono le differenze tra i modelli con diversi neighborhood sizes.

## Obiettivi dell'Analisi:
1. **Valutazione delle Performance**: Confronto delle metriche biologiche tra diversi neighborhood sizes
2. **Test Statistici**: Verifica della significatività statistica delle differenze
3. **Analisi delle Tendenze**: Identificazione di pattern e miglioramenti/peggioramenti
4. **Complessità Computazionale**: Analisi del costo computazionale vs. benefici
5. **Visualizzazioni Interattive**: Grafici Plotly per esplorazione approfondita


In [ ]:
import sys
import os
from pathlib import Path

# Add parent directory to path
# Get the directory where this notebook is located
notebook_dir = Path().absolute()
# Get the project root (parent of notebooks directory)
project_root = notebook_dir.parent
# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'experiments'))

# Import the analyzer
from experiments.analyze_neighborhood_sizes import NeighborhoodSizeAnalyzer

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("Imports completati!")
print(f"Project root: {project_root}")


## Configurazione

Definiamo i parametri per l'analisi:


In [ ]:
# Configurazione
# Use absolute paths based on project root
# If running from notebooks/, go up one level to project root
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

RESULTS_DIR = str(project_root / "experiments" / "results_extended")
HISTORIES_PATH = str(project_root / "histories.npy")
DEVICE = "auto"  # "auto", "cuda", "mps", or "cpu"
N_EVALUATIONS = 10  # Numero di valutazioni per modelli stocastici
NEIGHBORHOOD_SIZES = [3, 4, 5, 6, 7]
FORCE_RECOMPUTE = False  # Se True, rievaluta anche se CSV esistono

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Histories path: {HISTORIES_PATH}")
print(f"Device: {DEVICE}")
print(f"Neighborhood sizes: {NEIGHBORHOOD_SIZES}")
print(f"Number of evaluations: {N_EVALUATIONS}")
print(f"Paths exist: RESULTS_DIR={os.path.exists(RESULTS_DIR)}, HISTORIES={os.path.exists(HISTORIES_PATH)}")


## Inizializzazione dell'Analyzer

Creiamo l'istanza dell'analyzer e carichiamo/valutiamo i modelli:


In [ ]:
# Inizializza l'analyzer
analyzer = NeighborhoodSizeAnalyzer(
    results_dir=RESULTS_DIR,
    histories_path=HISTORIES_PATH,
    device=DEVICE,
    n_evaluations=N_EVALUATIONS
)

# Carica o valuta i modelli
analyzer.load_or_evaluate_models(
    neighborhood_sizes=NEIGHBORHOOD_SIZES,
    force_recompute=FORCE_RECOMPUTE
)

print("\n✓ Modelli caricati/valutati con successo!")


## Esplorazione dei Dati

Esaminiamo i dati delle metriche:


In [ ]:
# Parse delle metriche
df = analyzer.parse_metrics()

print("Shape del dataset:", df.shape)
print("\nColonne:", df.columns.tolist())
print("\nPrimi dati:")
df.head(10)


In [ ]:
# Statistiche descrittive per modello e neighborhood size
print("Statistiche descrittive per modello:")
print("="*60)
for model_type in df['Model Type'].unique():
    print(f"\n{model_type}:")
    model_data = df[df['Model Type'] == model_type]
    print(model_data.groupby('Neighborhood Size').agg(['mean', 'std']))


## Test Statistici

Eseguiamo test statistici per verificare la significatività delle differenze:


In [ ]:
# Esegui test statistici
stat_results = analyzer.statistical_tests()

# Visualizza risultati in modo più leggibile
import json
print("\n" + "="*60)
print("RISULTATI TEST STATISTICI")
print("="*60)

for metric, model_results in stat_results.items():
    print(f"\n{'='*60}")
    print(f"METRICA: {metric}")
    print(f"{'='*60}")
    for model_type, results in model_results.items():
        if 'kruskal_wallis' in results:
            kw = results['kruskal_wallis']
            significance = '***' if kw['p_value'] < 0.001 else '**' if kw['p_value'] < 0.01 else '*' if kw['p_value'] < 0.05 else '(non significativo)'
            print(f"\n  {model_type}:")
            print(f"    Kruskal-Wallis: H={kw['statistic']:.4f}, p={kw['p_value']:.6f} {significance}")
            
            if 'pairwise' in results and results['pairwise']:
                print(f"    Confronti a coppie significativi:")
                for pair, pair_result in results['pairwise'].items():
                    if pair_result['significant']:
                        nb1, nb2 = pair.split('_vs_')
                        sig = '***' if pair_result['p_value'] < 0.001 else '**' if pair_result['p_value'] < 0.01 else '*'
                        print(f"      NB{nb1} vs NB{nb2}: p={pair_result['p_value']:.6f} {sig}")


## Analisi delle Tendenze

Analizziamo le tendenze delle performance al variare del neighborhood size:


In [ ]:
# Analisi delle tendenze
trend_df = analyzer.performance_trend_analysis()

print("Analisi delle Tendenze:")
print("="*60)
trend_df


In [ ]:
# Visualizza miglioramenti/peggioramenti
print("\nMiglioramenti da NB=3 a NB=7:")
print("="*60)
improvements = trend_df[['Model Type', 'Metric', 'Improvement_3_to_7']].copy()
improvements = improvements.dropna()
improvements = improvements.sort_values('Improvement_3_to_7')

for _, row in improvements.iterrows():
    improvement = row['Improvement_3_to_7']
    direction = "MIGLIORAMENTO" if improvement > 0 else "PEGGIORAMENTO"
    print(f"{row['Model Type']} - {row['Metric']}: {improvement:.2f}% ({direction})")


## Analisi della Complessità Computazionale

Misuriamo il costo computazionale per ogni neighborhood size:


In [ ]:
# Analisi della complessità computazionale
complexity_df = analyzer.computational_complexity_analysis(n_samples=5)

print("Complessità Computazionale:")
print("="*60)
complexity_df


In [ ]:
# Visualizza complessità computazionale
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=complexity_df['Neighborhood Size'],
    y=complexity_df['Mean Time (s)'],
    mode='lines+markers',
    name='Tempo medio (s)',
    error_y=dict(type='data', array=complexity_df['Std Time (s)'], visible=True),
    line=dict(width=3, color='blue'),
    marker=dict(size=12)
))

fig.add_trace(go.Scatter(
    x=complexity_df['Neighborhood Size'],
    y=complexity_df['Normalized Time'],
    mode='lines+markers',
    name='Tempo normalizzato (vs NB=3)',
    line=dict(width=3, color='red', dash='dash'),
    marker=dict(size=12)
))

fig.update_layout(
    title='Complessità Computazionale vs Neighborhood Size',
    xaxis_title='Neighborhood Size',
    yaxis_title='Tempo (s) / Fattore di Normalizzazione',
    width=1000,
    height=600,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()


## Visualizzazioni Interattive

Creiamo visualizzazioni interattive con Plotly:


In [ ]:
# Crea tutte le visualizzazioni
analyzer.create_visualizations()

print("\n✓ Visualizzazioni create! Controlla la cartella analysis_plots/")


## Visualizzazioni Personalizzate nel Notebook

Creiamo visualizzazioni interattive direttamente nel notebook:


In [ ]:
# Dashboard interattiva con tutte le metriche
metric_cols = ['KL Divergence', 'Chi-Square', 'Categorical MMD', 
              'Tumor Size Diff', 'Border Size Diff', 'Spatial Variance Diff']

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=metric_cols,
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

colors = px.colors.qualitative.Set2
df_parsed = analyzer.parse_metrics()

for idx, metric in enumerate(metric_cols):
    if metric not in df_parsed.columns:
        continue
    
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    for model_idx, model_type in enumerate(df_parsed['Model Type'].unique()):
        model_data = df_parsed[df_parsed['Model Type'] == model_type]
        grouped = model_data.groupby('Neighborhood Size')[metric].agg(['mean', 'std'])
        
        sizes = grouped.index.values
        means = grouped['mean'].values
        stds = grouped['std'].values
        
        color = colors[model_idx % len(colors)]
        
        fig.add_trace(
            go.Scatter(
                x=sizes,
                y=means,
                mode='lines+markers',
                name=model_type if idx == 0 else '',
                line=dict(color=color, width=2),
                marker=dict(size=8, color=color),
                error_y=dict(type='data', array=stds, visible=True),
                showlegend=(idx == 0),
                hovertemplate=f'<b>{model_type}</b><br>' +
                            'Neighborhood Size: %{x}<br>' +
                            f'{metric}: %{{y:.4f}}<br>' +
                            '<extra></extra>'
            ),
            row=row, col=col
        )

fig.update_layout(
    title_text="Dashboard Completa: Performance per Neighborhood Size",
    height=1000,
    width=1800,
    font=dict(size=10),
    title_font_size=18,
    template='plotly_white'
)

fig.show()


In [ ]:
# Box plot interattivo per una metrica specifica
metric = 'KL Divergence'  # Cambia questa metrica per esplorare altre

fig = px.box(
    df_parsed, 
    x='Neighborhood Size', 
    y=metric, 
    color='Model Type',
    title=f'{metric} per Neighborhood Size e Tipo di Modello',
    labels={'Neighborhood Size': 'Neighborhood Size', metric: metric}
)

fig.update_layout(
    width=1200,
    height=700,
    font=dict(size=12),
    title_font_size=16,
    template='plotly_white'
)

fig.show()


## Analisi Costo-Beneficio

Confrontiamo il miglioramento delle performance con il costo computazionale:


In [ ]:
# Analisi costo-beneficio: miglioramento vs complessità
# Per ogni modello, calcoliamo il miglioramento relativo e lo confrontiamo con il costo

cost_benefit_analysis = []

for model_type in df_parsed['Model Type'].unique():
    model_data = df_parsed[df_parsed['Model Type'] == model_type]
    
    # Calcola miglioramento medio su tutte le metriche (normalizzato)
    nb3_data = model_data[model_data['Neighborhood Size'] == 3]
    nb7_data = model_data[model_data['Neighborhood Size'] == 7]
    
    if len(nb3_data) > 0 and len(nb7_data) > 0:
        improvements = []
        for metric in metric_cols:
            if metric in model_data.columns:
                mean3 = nb3_data[metric].mean()
                mean7 = nb7_data[metric].mean()
                if mean3 > 0:
                    improvement = (mean3 - mean7) / mean3 * 100  # % miglioramento
                    improvements.append(improvement)
        
        avg_improvement = np.mean(improvements) if improvements else 0
        
        # Complessità computazionale (normalizzata a NB=3)
        complexity_nb7 = complexity_df[complexity_df['Neighborhood Size'] == 7]['Normalized Time'].values[0] if len(complexity_df[complexity_df['Neighborhood Size'] == 7]) > 0 else 1
        
        cost_benefit_analysis.append({
            'Model Type': model_type,
            'Avg Improvement (%)': avg_improvement,
            'Computational Cost (x)': complexity_nb7,
            'Efficiency (Improvement/Cost)': avg_improvement / complexity_nb7 if complexity_nb7 > 0 else 0
        })

cost_benefit_df = pd.DataFrame(cost_benefit_analysis)
print("Analisi Costo-Beneficio (NB=3 vs NB=7):")
print("="*60)
cost_benefit_df


In [ ]:
# Visualizza analisi costo-beneficio
fig = go.Figure()

for _, row in cost_benefit_df.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Computational Cost (x)']],
        y=[row['Avg Improvement (%)']],
        mode='markers+text',
        name=row['Model Type'],
        marker=dict(size=15),
        text=[row['Model Type']],
        textposition="top center",
        hovertemplate=f"<b>{row['Model Type']}</b><br>" +
                      f"Miglioramento: {row['Avg Improvement (%)']:.2f}%<br>" +
                      f"Costo: {row['Computational Cost (x)']:.2f}x<br>" +
                      f"Efficienza: {row['Efficiency (Improvement/Cost)']:.2f}<br>" +
                      "<extra></extra>"
    ))

fig.update_layout(
    title='Analisi Costo-Beneficio: Miglioramento vs Complessità Computazionale',
    xaxis_title='Costo Computazionale (normalizzato a NB=3)',
    yaxis_title='Miglioramento Medio delle Performance (%)',
    width=1000,
    height=700,
    template='plotly_white',
    hovermode='closest'
)

# Aggiungi linee di riferimento
fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Nessun miglioramento")
fig.add_vline(x=1, line_dash="dash", line_color="gray", annotation_text="Costo base (NB=3)")

fig.show()


## Generazione Report Completo

Generiamo il report testuale completo:


In [ ]:
# Genera report completo
analyzer.generate_report()

print("\n✓ Report generato! Controlla il file neighborhood_size_analysis_report.txt")


## Conclusioni e Raccomandazioni

Sintesi dei risultati principali:


In [ ]:
# Trova la configurazione migliore per ogni metrica
print("="*60)
print("CONFIGURAZIONI MIGLIORI PER METRICA")
print("="*60)

for metric in metric_cols:
    if metric not in df_parsed.columns:
        continue
    
    best_idx = df_parsed[metric].idxmin()
    best_row = df_parsed.loc[best_idx]
    print(f"\n{metric}:")
    print(f"  Migliore: {best_row['Model Type']} con NB={best_row['Neighborhood Size']}")
    print(f"  Valore: {best_row[metric]:.4f}")

print("\n" + "="*60)
print("RACCOMANDAZIONI")
print("="*60)
print("""
1. Analizza i test statistici per determinare se le differenze sono significative
2. Considera il trade-off tra miglioramento delle performance e costo computazionale
3. Verifica se neighborhood sizes maggiori forniscono miglioramenti consistenti
4. Valuta se il miglioramento giustifica l'aumento del costo computazionale
5. Considera l'uso di neighborhood size maggiore solo se:
   - I test statistici mostrano differenze significative
   - Il miglioramento è consistente su tutte le metriche
   - Il costo computazionale è accettabile per il tuo caso d'uso
""")
